# 2D Fracture Voxelization and Analysis

In [1]:
import os
from subprocess import run
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

from scipy.ndimage import distance_transform_edt

from pysimfrac import SimFrac

## Helper functions

In [2]:
def tile_slice(frac, ind):
    """Tile Slice through the x-axis

    Parameters
    -------------
        frac : object
            Object with attributes 'top' and 'bottom' which are numpy arrays.
        ind : int
            Index or slice to repeat through the x-axis.

    Returns
    ------------
        None. The function modifies the 'top' and 'bottom' attributes of the `frac` object in-place.

    Notes
    ---------
        This function repeats a slice (ind) through the x-axis for the 'top' and 'bottom' attributes of the `frac` object.
    """
    frac.top = np.tile( frac.top[ind_slice], (frac.top.shape[1], 1))
    frac.bottom = np.tile( frac.bottom[ind_slice], (frac.top.shape[1], 1))

In [3]:
def calc_ap(frac_3D):
    """Calculate Mean Aperture of a Slice

    Parameters
    -------------
        frac_3D : np.array
            3D numpy array representing the fracture. The value '1' in the array indicates the presence of a fracture.

    Returns
    ------------
        float
            Mean aperture of the first slice. It is calculated as the fraction of cells with a fracture (value of 1) in the first slice.

    Notes
    ---------
        This function calculates the mean aperture of the first slice in the 3D array. It assumes all slices have the same aperture properties.
    """
    return np.sum(frac_3D[0,]==1)/frac_3D.shape[0]

In [4]:
def smallest_throat(frac_3D):
    """Calculate Size of the Smallest Constriction (Throat) 

    Parameters
    -------------
        frac_3D : np.array
            3D numpy array representing the fracture. 

    Returns
    ------------
        int
            Size of the smallest constriction or "throat" in the fracture.

    Notes
    ---------
        This function calculates the size of the smallest constriction, also known as a "throat", in the 3D array.
        It does so by multiplying the first slice of the array by a linear gradient and then counting the occurrences of unique values.
        A visualization of the multiplied slice is displayed using `plt.imshow`.
        An assertion is used to ensure that the fracture is connected by checking that the number of unique counts matches the slice size.
    """
    linear = np.arange( 0,frac_3D.shape[0] )*frac_3D[0,].T # multiplies frac times a linear gradient
    plt.imshow(linear)
    _, count = np.unique(linear, return_counts=True) # count the number of occurences of unique numbers
    assert count.size == frac_3D.shape[0] # this makes sure that it's connected
    return np.min(count)  # returns the size of the smallest "throat"

In [5]:
def voxelize_targetsize(self, size=128):

    """Voxelize the Fracture with a Target Size

    Parameters
    -------------
        size : int, optional
            Target size for the voxelized 3D array. Default is 128.

    Returns
    ------------
        None. The function modifies the 'frac_3D' attribute of the object in-place.

    Notes
    ---------
        This function voxelize the fracture based on a given target size. It ensures the following:
        1. There are no negative apertures by invoking the `aperture_check` method.
        2. Resets any negative values in the 'bottom' attribute to 0 using `reset_bottom`.
        3. Calculates the number of solid voxels needed and adjusts the z-dimension to ensure the fracture is lined.
        4. If the computed z-size exceeds certain thresholds, warnings or errors are raised.
        5. The fracture is then voxelized by iterating through its layers and adjusting the indices based on the calculated solid voxels.

        The resulting voxelized 3D array is stored in the 'frac_3D' attribute of the object.
    """
    
    import numpy.lib.index_tricks as ndi

    self.aperture_check()  # check that there are no negative apertures
    self.reset_bottom()  # push the negative values to 0
    solid_voxels = size - np.ceil(self.top.max()) # calculate how many voxels are needed
    z_size = np.ceil(self.top.max() + solid_voxels)  # add solid voxels so the fracture is lined

    assert z_size==size

    if z_size > 512:
        print_warning('The size of the array might be too big')
        if z_size > 2058:
            ValueError(f'Target size {z_size} is too big')

    # size of our 3D array
    size_3D = (*self.top.shape, int(z_size))
    flat_frac = np.zeros(np.prod(size_3D))

    # array of positional indices
    inds = np.arange(np.prod(size_3D)).reshape(size_3D)

    ind_top = np.ceil(self.top + solid_voxels // 2).astype(int)
    ind_bottom = np.ceil(self.bottom + solid_voxels // 2).astype(int)

    max_ap = (ind_top - ind_bottom).max()
    for i in range(max_ap):
      
        ind_layer = ind_bottom + i
        ind_layer[ind_layer > ind_top] = 0

        ind_1D = np.array([
            inds.T[ind_layer[I]][(I[1], I[0])]
            for I in ndi.ndindex(ind_layer.shape)
        ])
        flat_frac[ind_1D] = 1

    self.frac_3D = flat_frac.reshape(size_3D)
    self.frac_3D[:, :, 0] = 0

In [3]:
%%capture
myfrac = SimFrac( h = 0.01, lx = 1.28, ly = 1.28, method = "spectral")

myfrac.params['aniso']['value'] = 0.0
myfrac.params['H']['value'] = 0.7
myfrac.params['roughness']['value'] = 5
myfrac.params['mismatch']['value'] = 0.25
myfrac.params['model']['value'] = 'smooth'
myfrac.params['seed']['value'] = 1
myfrac.create_fracture()
#myfrac.voxelize_targetsize = voxelize_targetsize

# create a sheared fracture
myfrac2 = deepcopy(myfrac)
myfrac2.shear = 0.4
myfrac2.apply_shear()

TypeError: 'NoneType' object is not subscriptable

## Create 2D fracture surfaces

In [4]:
%%capture
ind_slice = 49 # tile along this index

# -> frac 1
myfrac.generate_2D(self, ind=ind_slice, mean_aperture=128//3)
#tile_slice(myfrac, ind_slice) # repeat the 49th slice across the x-axis
#myfrac.set_mean_aperture(128//3) 
myfrac.voxelize(target_size=128) 

# -> frac 2
#myfrac2.generate_2D(self, ind=49, mean_aperture=128//3)
#tile_slice(myfrac2, ind_slice) 
#myfrac2.set_mean_aperture(128//3) 
#myfrac2.voxelize_targetsize(myfrac2, size=128) 

AttributeError: 'SimFrac' object has no attribute 'generate_2D'

## Making sure that our 2D fractures are complaint

In [ ]:
min_throat = 20

In [ ]:
print(f'The mean aperture of the fracture 1 is {calc_ap(myfrac.frac_3D)}') 

In [ ]:
throat = smallest_throat(myfrac.frac_3D)
print(f'The smallest throat of fracture 1 is {throat}')
assert throat>min_throat

In [ ]:
print(f'The mean aperture of fracture 2 is {calc_ap(myfrac2.frac_3D)}')
throat = smallest_throat(myfrac2.frac_3D)
print(f'The smallest throat of the fracture 2 is {throat}')
assert throat>min_throat

In [ ]:
myfrac.set_mean_aperture(128//5); 
myfrac.voxelize_targetsize(myfrac, size=128);
print(f'The mean aperture of the fracture  is {calc_ap(myfrac.frac_3D)}')
throat = smallest_throat(myfrac.frac_3D)
print(f'The smallest throat of the fracture 3 is {throat}')
assert throat>min_throat

In [ ]:
plt.plot(myfrac.top[64,].squeeze().T, label='top')
plt.plot(myfrac.bottom[64,].squeeze().T,  label='bottom')
plt.plot(myfrac.top[64,].squeeze().T-myfrac.bottom[64,].squeeze().T,  label='aperture')
plt.title('Periodic Fracture')
plt.legend()